In [1]:
from gensim.models import Word2Vec

model_body = Word2Vec.load('word2vec_model')

In [5]:
print(model_body.wv["학교"])

[ 0.4592804   0.2094444  -0.05542162 -0.020831    0.26242614  0.01524775
 -0.7226699  -0.47200722 -0.0705606  -0.28813025  0.26643282 -0.03091559
  0.08962749 -0.21115255  0.16662134 -0.5519736  -0.20509414 -0.32254475
  0.64808726  0.05866559  0.5948888  -0.19594432 -0.2377142   0.08834521
  0.07252784 -0.04586444  0.3267818  -0.27102795 -0.07346851 -0.26353055
 -0.05979047  0.18799396 -0.25140342 -0.27716383  0.00930138  0.04287866
 -0.32841903  0.33082685 -0.51884    -0.060569   -0.25204003 -0.45604196
  0.4036232   0.2505236   0.14276965 -0.06402373 -0.3256995  -0.33293334
 -0.19913198  0.30167952  0.52976894  0.08050191 -0.15973462 -0.12455326
 -0.11670899 -0.24197766 -0.21457656  0.18805306 -0.06363223 -0.47736612
 -0.075086   -0.20496695 -0.2016432   0.24578448 -0.39946318 -0.16276109
  0.22354816  0.10503105  0.5524417  -0.08516693 -0.2043187   0.14267847
  0.45489725  0.34401459 -0.06306169 -0.23092678 -0.3558613   0.14150319
  0.24489789 -0.09227451 -0.33964393  0.4612065  -0

In [ ]:
from konlpy.tag import Komoran
tag = Komoran()
f = open('../Data/Preprocessed/body.txt', encoding="utf8")
sentences = f.read().strip().split('\n')
f.close()

error = []
result = []

for i, sentence in enumerate(sentences):
    try:
        result.append(tag.morphs(sentence))
    except:
        result.append(None)
        error.append(i)
        print("Error! ", i)
print(len(result))

In [ ]:
f = open('../Data/Preprocessed/Morphs/body.txt', "w", encoding="utf-8")
f.write(str(result))
f.close()

In [ ]:
f = open('../Data/Preprocessed/reply.txt', encoding="utf-8")
sentences = [sentence[0]['text'] for sentence in list(map(eval, f.read().strip().split("\n")))]
f.close()

error_reply = []
result_reply = []

for i, sentence in enumerate(sentences):
    try:
        result_reply.append(tag.morphs(sentence))
    except:
        result_reply.append(None)
        error_reply.append(i)
        print("Error! ", i)
print(len(result_reply))

In [ ]:
error_total = error+error_reply
error_total.sort(reverse=True)

for error_idx in error_total:
    del result[error_idx]
    del result_reply[error_idx]

In [ ]:
f = open('../Data/Preprocessed/Morphs/body-noerror.txt', "w", encoding="utf-8")
f.write(str(result))
f.close()
f = open('../Data/Preprocessed/Morphs/reply-noerror.txt', "w", encoding="utf-8")
f.write(str(result_reply))
f.close()

In [2]:
### Error가 제거된 데이터 Read

f = open('../Data/Preprocessed/Morphs/body-noerror.txt', 'r', encoding='utf-8')
result = eval(f.read())
f.close()

f = open('../Data/Preprocessed/Morphs/reply-noerror.txt', 'r', encoding='utf-8')
result_reply = eval(f.read())
f.close()

In [3]:
print(len(result))
print(len(result_reply))

4357
4357


In [4]:
import numpy as np

### Make vector list to all text of each words

x_data, y_data = [], []
result_len = len(result)

for sentence_idx in range(result_len):
    vector_x, vector_y = [], []
    
    for word in result[sentence_idx]:
        try:
            vector_x.append(model_total.wv[word])
        except:
            pass
        
    for word in result_reply[sentence_idx]:
        try:
            vector_y.append(model_total.wv[word])
        except:
            pass
        
    x_data.append(vector_x)
    y_data.append(vector_y)
    
x_data = np.array(x_data)
y_data = np.array(y_data)

In [5]:
### shape (# of data, # of word, length of embedding vector)

print("x data shape: ", x_data.shape)
print("y data shape: ", y_data.shape)

x data shape:  (4357,)
y data shape:  (4357,)


In [6]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess= tf.Session(config=config)

In [7]:
import numpy as np

#for y in y_data[0]:
#    print(model_total.wv.most_similar(positive=[y], topn=1))
x_data_maxlen = 500
y_data_maxlen = 50

from keras.preprocessing import sequence
from keras import backend as K
x_data = sequence.pad_sequences(x_data, padding='post', maxlen=x_data_maxlen, dtype=K.floatx())
y_data = sequence.pad_sequences(y_data, padding='pre', maxlen=y_data_maxlen, dtype=K.floatx())

Using TensorFlow backend.


In [18]:
print(x_data.shape)
print(y_data.shape)

print(x_data[0][325])

(4357, 500, 100)
(4357, 50, 100)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Dropout, Input
from keras.layers import LSTM, SimpleRNN, GRU
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.optimizers import RMSprop

input_shape = x_train.shape[1:]
target_shape = y_train.shape[1:]
latent_dim = 128

encoder_inputs = Input(shape=input_shape)
#encoder_inputs = BatchNormalization()(encoder_inputs)
encoder = LSTM(latent_dim, return_sequences=True, return_state=True, kernel_initializer='zeros', recurrent_initializer='zeros')
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=target_shape)
#decoder_inputs = BatchNormalization()(decoder_inputs)
decoder = LSTM(latent_dim, return_sequences=True, return_state=True, kernel_initializer='zeros', recurrent_initializer='zeros')
decoder_outputs, _, _ = decoder(decoder_inputs, initial_state = encoder_states)
decoder_dense = Dense(target_shape[1], activation='tanh')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=decoder_outputs)
rmsprop = RMSprop()

In [ ]:
model.compile(loss='hinge', optimizer=rmsprop, metrics=['accuracy'])
print(model.summary())

In [ ]:
################################ 바꿔가면서 해보세요 ######################################
num_epochs = 20
mini_batch_size = 64
######################################################################################
history = model.fit([x_data, y_data], y_data, epochs=num_epochs, batch_size=mini_batch_size, verbose=1, validation_split=0.1)

In [ ]:
idx = 50
b = model.predict([x_data[idx:idx+1], np.zeros_like(y_data[0:1])])
for i in range(10):
    print(model_total.wv.most_similar(positive=[y_data[idx][i]], topn=1))
print()
for i in range(10):
    print(model_total.wv.most_similar(positive=[b[0][i]], topn=1))